In [1]:
# Imports go here
from granite_io.io.query_expansion import QueryExpansionIOProcessor
from granite_io.io.query_rewrite import QueryRewriteIOProcessor
from granite_io.io.granite_3_2.input_processors.granite_3_2_input_processor import (
    Granite3Point2Inputs,
)
from granite_io.backend.vllm_server import LocalVLLMServer
from granite_io import make_io_processor, make_backend
from granite_io.io.base import ChatCompletionInputs, ChatCompletionResults

In [2]:
# Constants go here
base_model_name = "ibm-granite/granite-3.2-8b-instruct"
lora_model_name = "ibm-granite/granite-3.2-8b-lora-rag-query-rewrite"
run_server = False

In [ ]:
if run_server:
    # Start by firing up a local vLLM server and connecting a backend instance to it.
    server = LocalVLLMServer(
        base_model_name, lora_adapters=[(lora_model_name, lora_model_name)]
    )
    server.wait_for_startup(200)
    query_rewrite_lora_backend = server.make_lora_backend(lora_model_name)
    backend = server.make_backend()
else:  # if not run_server
    # Use an existing server.
    # Modify the constants here as needed.
    openai_base_url = "http://localhost:55555/v1"
    openai_api_key = "granite_intrinsics_1234"
    openai_base_model_name = base_model_name
    openai_lora_model_name = lora_model_name
    
    backend = make_backend(
        "openai",
        {
            "model_name": openai_base_model_name,
            "openai_base_url": openai_base_url,
            "openai_api_key": openai_api_key,
        },
    )
    query_rewrite_lora_backend = make_backend(
        "openai",
        {
            "model_name": openai_lora_model_name,
            "openai_base_url": openai_base_url,
            "openai_api_key": openai_api_key,
        },
    )

In [4]:
input_messages = [
    {
        "role": "assistant",
        "content": "Welcome to the California State Parks help desk.",
    },
    {
        "role": "user",
        "content": "I'm a student. Do you have internships?",
    },
    {
        "role": "assistant",
        "content": "The California State Parks hires Student Assistants "
        "to perform a variety of tasks that require limited or no previous "
        "work experience.",
    },
    {
        "role": "user", 
        "content": "Cool, how do I sign up?"
    },
]

chat_input_tmp = ChatCompletionInputs(messages=input_messages, generate_inputs={"temperature": 0.0,"max_tokens": 4096,})
print("Inputs for chat completion:", chat_input_tmp)



Inputs for chat completion: messages=[AssistantMessage(content='Welcome to the California State Parks help desk.', role='assistant', tool_calls=[], reasoning_content=None, citations=None, documents=None, hallucinations=None, stop_reason=None), UserMessage(content="I'm a student. Do you have internships?", role='user'), AssistantMessage(content='The California State Parks hires Student Assistants to perform a variety of tasks that require limited or no previous work experience.', role='assistant', tool_calls=[], reasoning_content=None, citations=None, documents=None, hallucinations=None, stop_reason=None), UserMessage(content='Cool, how do I sign up?', role='user')] tools=[] generate_inputs=GenerateInputs(prompt=None, model=None, best_of=None, echo=None, frequency_penalty=None, logit_bias=None, logprobs=None, max_tokens=4096, n=None, presence_penalty=None, stop=None, stream=None, stream_options=None, suffix=None, temperature=0.0, top_p=None, user=None, extra_headers=None, extra_body={})

In [5]:
# Spin up an IO processor for the base model
io_processor = make_io_processor(base_model_name, backend=backend)
rewrite_io_proc = QueryRewriteIOProcessor(query_rewrite_lora_backend)

In [6]:
rag_io_proc = QueryExpansionIOProcessor(
    backend,
    io_processor,
    rewrite_io_proc,
)

qe_result = rag_io_proc.create_chat_completion(chat_input_tmp)
print(qe_result)

qe_result_strs = [r.next_message.content for r in qe_result.results]
print("\nQuery Expansion Results:")
for i, result in enumerate(qe_result_strs):
    print(f"Result {i + 1}: {result}")


QUERY REWRITE: results=[ChatCompletionResult(next_message=UserMessage(content='How do I sign up for the Student Assistant program at California State Parks?', role='user'))]
SYNONMOUS QUERY: results=[GenerateResult(completion_string='How can I apply for the California State Parks Student Assistant position?', completion_tokens=[], stop_reason='stop')]
ENRICHED QUERIES: results=[GenerateResult(completion_string='\n1. California State Parks Student Internship Program\n2. Apply for California State Parks Student Assistant position\n3. Requirements for California State Parks Student Assistant\n4. California State Parks Student Assistant application\n5. How to apply for California State Parks Student Internship\n6. Eligibility criteria for California State Parks Student Assistant\n7. California State Parks Student Assistant job description\n8. Steps to apply for California State Parks Student Internship\n9. Prerequisites for California State Parks Student Assistant\n10. California State Par